In [10]:
from google.colab import files
files.upload()
!ls

Saving iris.data to iris.data
iris3.data  iris.data  iristest.data  sample_data


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [11]:
train_dataset = pd.read_csv('iris.data').replace(['Iris-setosa','Iris-versicolor','Iris-virginica'], [0,1,2])
print(f'Rows: {len(train_dataset)}')
print(f'Columns: {len(train_dataset.columns)}')
train_dataset.head()

test_dataset = pd.read_csv('iristest.data').replace(['Iris-setosa','Iris-versicolor','Iris-virginica'], [0,1,2])
print(f'Rows: {len(test_dataset)}')
print(f'Columns: {len(test_dataset.columns)}')
test_dataset.head()

Rows: 120
Columns: 5
Rows: 30
Columns: 5


,sepal_length,sepal_width,petal_length,petal_width,class
0,5.0,3.5,1.3,0.3,0
1,4.5,2.3,1.3,0.3,0
2,4.4,3.2,1.3,0.2,0
3,5.0,3.5,1.6,0.6,0
4,5.1,3.8,1.9,0.4,0


In [12]:
# CART on the Bank Note dataset
from random import randrange
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, class_values):
	gini = 0.0
	for class_value in class_values:
		for group in groups:
			size = len(group)
			if size == 0:
				continue
			proportion = [row[-1] for row in group].count(class_value) / float(size)
			gini += (proportion * (1.0 - proportion))
	return gini
 
# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups; print(b_index, '-', b_score)
	return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root
 
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']
 
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)
 
# Print a decision tree
def print_tree(node, depth=0):
  if isinstance(node, dict):
    print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
    print_tree(node['left'], depth+1)
    print_tree(node['right'], depth+1)
  else:
    print('%s[%s]' % ((depth*' ', node)))

data = np.array(train_dataset).tolist()

# evaluate algorithm
n_folds = 2
max_depth = 2
min_size = 2
scores = evaluate_algorithm(data, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

0 - 0.7575424840200881
0 - 0.6835538903061225
0 - 0.6330589849108368
0 - 0.6251077276644643
0 - 0.61374154503382
0 - 0.610207100591716
2 - 0.5321795565170363
2 - 0.4907725893778109
2 - 0.42240587695133147
0 - 0.0
0 - 0.6487603305785125
0 - 0.42240587695133147
1 - 0.404296875
2 - 0.35777777777777775
2 - 0.26317580340264646
0 - 0.9535147392290251
0 - 0.5754458161865569
2 - 0.5639017443930224
2 - 0.5031236984589754
2 - 0.4617473970122228
0 - 0.0
0 - 0.4854253420582987
0 - 0.47417355371900827
0 - 0.47012345679012346
0 - 0.4659735349716446
0 - 0.4617473970122228
0 - 0.4338842975206612
0 - 0.42184964845862627
2 - 0.39287688622264083
2 - 0.34569254699124824
2 - 0.33480552885974335
2 - 0.33240997229916897
3 - 0.23789509625390223
Scores: [91.66666666666666, 93.33333333333333]
Mean Accuracy: 92.500%


In [13]:
test_data = np.array(test_dataset).tolist()
tree = build_tree(test_data, max_depth, min_size)
print_tree(tree)
print(tree)

count = 0
for row in test_data:
 prediction = predict(tree, row)
 print('Expected=%d, Got=%d' % (row[-1], prediction))
 if prediction == row[-1]:
   count+=1
print(count)

0 - 0.650887573964497
0 - 0.64
0 - 0.6049149338374291
2 - 0.5442176870748299
2 - 0.5
0 - 0.0
0 - 0.49382716049382713
0 - 0.35502958579881655
2 - 0.2777777777777778
2 - 0.16528925619834714
2 - 0.0
[X3 < 3.000]
 [X1 < 5.000]
  [0.0]
  [0.0]
 [X3 < 5.000]
  [1.0]
  [2.0]
{'index': 2, 'value': 3.0, 'left': {'index': 0, 'value': 5.0, 'left': 0.0, 'right': 0.0}, 'right': {'index': 2, 'value': 5.0, 'left': 1.0, 'right': 2.0}}
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
Expected=2, Got=2
30
